In [3]:
import numpy as np
import pandas as pd 
import tensorflow as tf

In [4]:
from glob import glob
from tqdm import tqdm_notebook as tqdm

In [12]:
train_recs = sorted(glob('data/train*.tfrecord'))
val_recs = sorted(glob('data/validate*.tfrecord'))
test_recs = sorted(glob('data/test*.tfrecord'))

In [13]:
def read_file(fine_name):
    vid_ids = []
    labels = []
    mean_rgb = []
    mean_audio = []

    for example in tf.python_io.tf_record_iterator(fine_name):
        tf_example = tf.train.Example.FromString(example)

        features = tf_example.features.feature

        id = features['video_id'].bytes_list.value[0].decode(encoding='UTF-8')
        label = np.array(features['labels'].int64_list.value, dtype='uint16')
        rgb = np.array(features['mean_rgb'].float_list.value, dtype='float32')
        audio = np.array(features['mean_audio'].float_list.value, dtype='float32')
                
        vid_ids.append(id)
        labels.append(label)
        mean_rgb.append(rgb)
        mean_audio.append(audio)

    return vid_ids, labels, mean_rgb, mean_audio

In [14]:
import traceback

In [15]:
def read_data(files_list):
    vid_ids = []
    labels = []
    mean_rgb = []
    mean_audio = []

    for f in files_list:
        for example in tf.python_io.tf_record_iterator(f):
            good = False
    
            try:
                tf_example = tf.train.Example.FromString(example)

                features = tf_example.features.feature

                id = features['video_id'].bytes_list.value[0].decode(encoding='UTF-8')
                label = np.array(features['labels'].int64_list.value, dtype='uint16')
                rgb = np.array(features['mean_rgb'].float_list.value, dtype='float32')
                audio = np.array(features['mean_audio'].float_list.value, dtype='float32')
                good = True
            except:
                print('exception ocurred while reading, swallowing it')
                traceback.print_exc()
                continue

            if good:
                vid_ids.append(id)
                labels.append(label)
                mean_rgb.append(rgb)
                mean_audio.append(audio)

    return vid_ids, labels, mean_rgb, mean_audio

In [16]:
train_ids, train_labs, train_rgb, train_audio = read_data(train_recs[:10])

In [17]:
X_10_concat = np.hstack([train_rgb, train_audio])

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_10_concat)

In [19]:
X_10_concat = scaler.transform(X_10_concat)

In [20]:
X_10_concat.shape

(11824, 1152)

In [21]:
val_ids, val_labs, val_rgb, val_audio = read_data(val_recs[:100])

In [22]:
X_100_val = np.hstack([val_rgb, val_audio])
X_100_val = scaler.transform(X_100_val)

In [25]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU, ELU, LeakyReLU
from keras.callbacks import EarlyStopping, Callback

Using TensorFlow backend.


In [27]:
input_dim = X_10_concat.shape[1]
output_dim = 4716# len(label_names)

In [29]:
def to_dense_lab(labs):
    res = np.zeros(output_dim, dtype='uint8')
    res[labs] = 1
    return res

def to_dense_labs(list_labs):
    return np.array([to_dense_lab(l) for l in list_labs])

In [30]:
val_labs_dense = to_dense_labs(val_labs)

In [31]:
from sklearn.metrics import log_loss, f1_score

In [32]:
class WatchlistCallback(Callback):
    def __init__(self, watchlist):
        super(Callback, self).__init__()
        self.X, self.y = watchlist

    def on_epoch_end(self, epoch, logs={}):
        if epoch % 5 == 0:
            self.print_score(epoch)

    def print_score(self, epoch):
        y_pred = self.model.predict(self.X, verbose=0)

        print("epoch no %d" % (epoch), end=', ')
        print('logloss=%.4f' % log_loss(self.y, y_pred), end=', ')

        #f = f1_score(self.y, y_pred >= 0.5, average='micro')
        #print('f1=%0.4f' % f, end=', ')
        print()

In [33]:
from sklearn.utils import shuffle

In [34]:
#watchlist = WatchlistCallback(watchlist=(X_100_val_pca, val_labs_dense))
watchlist = WatchlistCallback(watchlist=(X_100_val, val_labs_dense))

In [35]:
def prepare_batches(seq, n):
    for i in range(0, len(seq), n):
        yield seq[i:i+n]

def image_gen(file_list, n=2, seed=0):
    i = seed + 0

    while True:
        files = shuffle(file_list, random_state=i)

        batches = prepare_batches(files, n)

        for batch in batches:
            _, labs, rgb, audio = read_data(batch)
            X = np.hstack([rgb, audio])
            X = scaler.transform(X)
            #X = pca.transform(X)
            y = to_dense_labs(labs)

            yield X, y

        i = i + 1

In [38]:
from keras_tqdm import TQDMNotebookCallback

In [39]:
#input_dim = X_100_val_pca.shape[1]
input_dim = X_100_val.shape[1]

In [40]:
model = Sequential()

model.add(Dense(input_dim=input_dim, units=1500, kernel_initializer='glorot_uniform')) 
model.add(PReLU())
model.add(Dropout(0.5))

model.add(BatchNormalization())

model.add(Dense(units=output_dim, kernel_initializer='glorot_uniform')) 
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01))

In [41]:
gen = image_gen(train_recs, n=8, seed=2)


In [42]:
model.fit_generator(gen, steps_per_epoch=32, epochs=100, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

KeyboardInterrupt: 

In [ ]:
model.fit_generator(gen, steps_per_epoch=32, epochs=100, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

In [ ]:
model.fit_generator(gen, steps_per_epoch=32, epochs=100, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

In [ ]:
model.fit_generator(gen, steps_per_epoch=32, epochs=100, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

In [241]:
model.fit_generator(gen, steps_per_epoch=1, epochs=1, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

epoch no 0, logloss=8.8633, f1=0.7291



In [ ]:
model.fit_generator(gen, steps_per_epoch=32, epochs=100, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

epoch no 44, logloss=9.8677, f1=0.6606

epoch no 45, logloss=9.8835, f1=0.6621

epoch no 46, logloss=9.8794, f1=0.6655

epoch no 47, logloss=9.8846, 

KeyboardInterrupt: 

In [ ]:
model.fit_generator(gen, steps_per_epoch=32, epochs=100, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

In [187]:
model.fit_generator(gen, steps_per_epoch=32, epochs=100, verbose=0, 
                    callbacks=[watchlist, TQDMNotebookCallback()])

epoch no 0, logloss=8.9754, f1=0.7192
epoch no 1, logloss=8.9735, f1=0.7207
epoch no 2, logloss=8.9796, f1=0.7211
epoch no 3, logloss=8.9760, f1=0.7219
epoch no 4, logloss=8.9635, f1=0.7196
epoch no 5, logloss=8.9611, f1=0.7225
epoch no 6, logloss=8.9615, f1=0.7199
epoch no 7, logloss=8.9581, f1=0.7231
epoch no 8, logloss=8.9675, f1=0.7166
epoch no 9, logloss=8.9855, f1=0.7244
epoch no 10, logloss=8.9567, f1=0.7215
epoch no 11, logloss=8.9587, f1=0.7224
epoch no 12, logloss=8.9536, f1=0.7223
epoch no 13, logloss=8.9541, f1=0.7207
epoch no 14, logloss=8.9513, f1=0.7209
epoch no 15, logloss=8.9409, f1=0.7215
epoch no 16, logloss=8.9387, f1=0.7201
epoch no 17, logloss=8.9412, f1=0.7218
epoch no 18, logloss=8.9368, f1=0.7216
epoch no 19, logloss=8.9490, f1=0.7226
epoch no 20, logloss=8.9425, f1=0.7217
epoch no 21, 

KeyboardInterrupt: 

In [149]:
from comp_metrics import AveragePrecisionCalculator
calc = AveragePrecisionCalculator(top_n=20)
ap = calc.ap_at_n

def gap(pred, actual):
    all = []
    for i in range(len(pred)):
        v = ap(pred[i], actual[i])
        all.append(v)
    return np.mean(all)

In [242]:
pred = model.predict(X_100_val)

In [157]:
gap(pred[:200], val_labs_dense[:200])

0.76565193594657965

In [243]:
gap(pred[:200], val_labs_dense[:200])

0.78129405906202265

In [136]:
%load_ext autoreload
%autoreload 2

In [182]:
def prepare_pred_row(prow):
    classes = (-prow).argsort()[:20]
    scores = prow[classes]
    return ' '.join(['%d %0.3f' % (c, s) for (c, s) in zip(classes, scores)])

In [244]:
with open('subm2.csv', 'w') as f:
    f.write('VideoId,LabelConfidencePairs\n')

    for fn in tqdm(test_recs):
        ids, _, rgb, audio = read_file(fn)
        X = np.hstack([rgb, audio])
        X = scaler.transform(X)

        pred = model.predict(X)

        for id, prow in zip(ids, pred):
            lab_conf = prepare_pred_row(prow)
            f.write('%s,%s\n' % (id, lab_conf))

        #break

In [245]:
!gzip subm2.csv

In [246]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 2500)              2882500   
_________________________________________________________________
p_re_lu_26 (PReLU)           (None, 2500)              2500      
_________________________________________________________________
dropout_26 (Dropout)         (None, 2500)              0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 2500)              10000     
_________________________________________________________________
dense_39 (Dense)             (None, 4716)              11794716  
_________________________________________________________________
activation_12 (Activation)   (None, 4716)              0         
Total params: 14,689,716.0
Trainable params: 14,684,716.0
Non-trainable params: 5,000.0
______________________________________________________